# Simple Harmonic Motion
## Analysis of Multiple Fits

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.optimize import curve_fit
# Set the number of digits to be printed after the decimal point
np.set_printoptions(precision=4)

In [ ]:
df1 = pd.read_csv("273_Lab4_100_no_plate.csv")

print(list(df1.columns))
substrings_to_delete = ["Ping Echo", "Date and Time Run"]
columns_to_drop = [
    col for col in df1.columns
    if any(sub in col for sub in substrings_to_delete)
]
df1.drop(columns=columns_to_drop, inplace=True)

# rename first csv column numbers
rename_mapping = {}
# The runs we want to change go from 9 to 28 (20 runs total)
for old_run_num in range(9, 29):
    new_run_num = old_run_num - 8 # Renumbers 9 to 1, 10 to 2, ..., 28 to 20

    # The pattern to find the old run number in the column header
    # e.g., looks for ' #9' at the end of the string
    old_pattern = f' #{old_run_num}'

    # The string to replace it with
    new_string = f' #{new_run_num}'

    # Iterate through all current column names
    for col in df1.columns:
        # Check if the column name ends with the old run pattern
        if col.endswith(old_pattern):
            # Create the new column name
            new_col = col.replace(old_pattern, new_string)

            # Add the mapping to the dictionary
            rename_mapping[col] = new_col

# 2. Rename the columns using the mapping
df1 = df1.rename(columns=rename_mapping)

print(list(df1.columns))

In [ ]:
df2 = pd.read_csv("273_Lab4_100_small_plate.csv")
df3 = pd.read_csv("273_Lab4_250_no_plate.csv")
df4 = pd.read_csv("273_Lab4_250_small_plate.csv")
df5 = pd.read_csv("273_Lab4_250_large_plate.csv")

dfArr = [df1, df2, df3, df4, df5]
plotTitles = ["100_no_plate", "100_small_plate", "250_no_plate", "250_small_plate", "250_large_plate"]

In [ ]:
m1 = 102.4 # 100g no plate
m2 = 110.9 # 100g small plate

m3 = 253.4 # 250g no plate
m4 = 256.9 # 250g small plate
m5 = 268.2 # 250g large plate

Check that the format of the file is as expected, the top row should contain the title of the column, for example: "Time (s) Run #1", "Position (m) Run #1", etc. The columns of interest are Time and Position and if you have multiple runs in the file these should correspond to different run numbers. *This example expects 20 runs in the input file.*

## Reminder on how to obtain fit values and uncertainties using Scipy curve_fit
#### Documentation at https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html
#### More information about curve fitting at https://python4mpia.github.io/fitting_data/least-squares-fitting.html
#### First define a damped sine function
### $$A e^{-Bt} \sin(Wt+P) + C$$
Parameters are: amplitude (A), damping (B), frequency (W), phase (P) and offset (C)

In [ ]:
def my_sin(x, amplitude, damp, freq, phase, offset):
    return amplitude * np.exp(-damp * x) * np.sin(x * freq + phase) + offset


### Statistical analysis of the fit results of 20 repeated experiments
Perform fit from first to last specified data point. Note that data is collected at 20 Hz, i.e. a new position measurement is made every 50 ms.

For the position measurement error, the Pasco motion sensor has resolution of 1 mm.

Store the fit parameters in an array.



In [ ]:
A = []
B = []
W = []
P = []
C = []

guess_damp = 0.01
guess_freq = 5
guess_phase = -3

# first and last points used in the fit
first = 0
last = first+600


for i in range(0, len(dfArr)):
  df = dfArr[i]
  plot_title = plotTitles[i]
  for i in range(19):
    x=df["Time (s) Run #"+str(i+1)].dropna()
    x=x.to_numpy()
    x=x.transpose()
    y=df["Position (m) Run #"+str(i+1)].dropna()
    y=y.to_numpy()
    y=y.transpose()

    guess_amplitude = np.max(y)-np.mean(y)
    guess_offset = np.mean(y)
    p0_guess=[guess_amplitude, guess_damp, guess_freq, guess_phase, guess_offset]

    dy = np.full(len(x),0.001)
    fit = curve_fit(my_sin, x[first:last], y[first:last], sigma=dy[first:last], p0=p0_guess, absolute_sigma=True)

    # Extract fit results: parameters
    param = fit[0]
    A.append(param[0])
    B.append(param[1])
    W.append(param[2])
    P.append(param[3])
    C.append(param[4])


      
  plt.figure(figsize=(6,4))
  # Specify which parameter to plot and the axis label
  parameter = W
  label_title = 'Amplitude (m)'

  lowx = np.mean(parameter)-5*np.std(parameter)
  highx = np.mean(parameter)+5*np.std(parameter)
  plt.hist(parameter, bins=15, range=(lowx, highx))
  plt.xlabel(label_title)
  plt.ylabel('Number of experiments')
  plt.suptitle(plot_title)
  plt.show()
  print('Avg   ' + "{:.5}".format(np.mean(parameter)) + " +/- " + "{:.2}".format(np.std(parameter)/np.sqrt(len(parameter))) + ", std dev " + "{:.3}".format(np.std(parameter)))

### Histogram the measurements
Specify which parameter to plot and the axis label. *Don't forget to add the units.*

In [ ]:

# plt.figure(figsize=(6,4))
# # Specify which parameter to plot and the axis label
# parameter = W
# label_title = 'Amplitude (m)'

# lowx = np.mean(parameter)-5*np.std(parameter)
# highx = np.mean(parameter)+5*np.std(parameter)
# plt.hist(parameter, bins=15, range=(lowx, highx))
# plt.xlabel(label_title)
# plt.ylabel('Number of experiments')
# plt.suptitle(plot_title)
# plt.show()
# print('Avg   ' + "{:.5}".format(np.mean(parameter)) + " +/- " + "{:.2}".format(np.std(parameter)/np.sqrt(len(parameter))) + ", std dev " + "{:.3}".format(np.std(parameter)))

### Real Stuff
